##  Calculate Yearly DPLE variables

In [1]:
import xarray as xr
import numpy as np
import metabolic as mi
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """

    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)

In [3]:
USER = os.environ['USER']

In [4]:
path = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{path}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly.nc'
filename = f'{path}/DPLE_driftcorrected_Phi_Eo_space_vertical_mean_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)

In [5]:
dsf = xr.Dataset()
for cc in range(13):
    for layer in ['0-200m', '200-600m']:
        dsf[f'Phi_{cc}_{layer}'] = xr_reshape(ds[f'Phi_{cc}_{layer}'][:, 2:], 'L', ['year', 'mon'], [np.arange(10), np.arange(12)]).mean(dim='mon')

In [6]:
USER = os.environ['USER']
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)

In [7]:
%%time
dsf.load()

CPU times: user 1.87 ms, sys: 3.9 ms, total: 5.78 ms
Wall time: 28.2 ms


<xarray.Dataset>
Dimensions:          (nlat: 140, nlon: 195, Y: 64, year: 10)
Coordinates:
    ULONG            (nlat, nlon) float64 101.7 102.9 104.0 ... 319.5 320.0
    ULAT             (nlat, nlon) float64 15.13 15.13 15.14 ... 70.63 70.63
  * Y                (Y) int64 1954 1955 1956 1957 1958 ... 2014 2015 2016 2017
    TLAT             (nlat, nlon) float64 14.96 14.96 14.97 ... 70.42 70.42
    TLONG            (nlat, nlon) float64 101.2 102.3 103.4 ... 319.2 319.7
  * year             (year) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: nlat, nlon
Data variables: (12/26)
    Phi_0_0-200m     (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan
    Phi_0_200-600m   (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan
    Phi_1_0-200m     (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan
    Phi_1_200-600m   (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan
    Phi_2_0-200m     (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan
    Phi_2_200-600m   (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan
    ...               ...
    Phi_10_0-200m    (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan
    Phi_10_200-600m  (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan
    Phi_11_0-200m    (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan
    Phi_11_200-600m  (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan
    Phi_12_0-200m    (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan
    Phi_12_200-600m  (Y, year, nlat, nlon) float64 nan nan nan ... nan nan nan

In [8]:
#dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly_to_yearly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_Eo_space_vertical_mean_ens_mean_monthly_to_yearly.nc', mode='w')

## LME monthly to yearly

In [9]:
path = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{path}/DPLE_driftcorrected_Phi_LME_ens_mean_monthly.nc'
filename = f'{path}/DPLE_driftcorrected_Phi_Eo_space_LME_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)
ds

<xarray.Dataset>
Dimensions:          (lme: 11, Y: 64, L: 122)
Coordinates:
  * lme              (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * Y                (Y) int64 1954 1955 1956 1957 1958 ... 2014 2015 2016 2017
  * L                (L) int64 1 2 3 4 5 6 7 8 ... 116 117 118 119 120 121 122
Data variables: (12/26)
    Phi_0_0-200m     (lme, Y, L) float64 ...
    Phi_0_200-600m   (lme, Y, L) float64 ...
    Phi_1_0-200m     (lme, Y, L) float64 ...
    Phi_1_200-600m   (lme, Y, L) float64 ...
    Phi_2_0-200m     (lme, Y, L) float64 ...
    Phi_2_200-600m   (lme, Y, L) float64 ...
    ...               ...
    Phi_10_0-200m    (lme, Y, L) float64 ...
    Phi_10_200-600m  (lme, Y, L) float64 ...
    Phi_11_0-200m    (lme, Y, L) float64 ...
    Phi_11_200-600m  (lme, Y, L) float64 ...
    Phi_12_0-200m    (lme, Y, L) float64 ...
    Phi_12_200-600m  (lme, Y, L) float64 ...

In [10]:
dsf = xr.Dataset()
for cc in range(13):
    for layer in ['0-200m', '200-600m']:
        dsf[f'Phi_{cc}_{layer}'] = xr_reshape(ds[f'Phi_{cc}_{layer}'][:, :, 2:], 'L', ['year', 'mon'], [np.arange(10), np.arange(12)]).mean(dim='mon')

In [11]:
USER = os.environ['USER']
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)

In [12]:
%%time
dsf.load()

CPU times: user 1.07 ms, sys: 48 µs, total: 1.12 ms
Wall time: 1.12 ms


<xarray.Dataset>
Dimensions:          (lme: 11, Y: 64, year: 10)
Coordinates:
  * lme              (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * Y                (Y) int64 1954 1955 1956 1957 1958 ... 2014 2015 2016 2017
  * year             (year) int64 0 1 2 3 4 5 6 7 8 9
Data variables: (12/26)
    Phi_0_0-200m     (lme, Y, year) float64 0.6116 0.6204 ... 0.6245 0.6278
    Phi_0_200-600m   (lme, Y, year) float64 0.1229 0.1223 ... 0.08687 0.08616
    Phi_1_0-200m     (lme, Y, year) float64 0.716 0.7241 ... 0.7083 0.7114
    Phi_1_200-600m   (lme, Y, year) float64 0.1427 0.1419 ... 0.1003 0.09947
    Phi_2_0-200m     (lme, Y, year) float64 0.8384 0.8453 ... 0.8036 0.8062
    Phi_2_200-600m   (lme, Y, year) float64 0.1656 0.1648 ... 0.1158 0.1148
    ...               ...
    Phi_10_0-200m    (lme, Y, year) float64 2.998 2.951 2.904 ... 2.217 2.207
    Phi_10_200-600m  (lme, Y, year) float64 0.5496 0.5455 ... 0.3666 0.3627
    Phi_11_0-200m    (lme, Y, year) float64 3.521 3.455 3.393 ... 2.519 2.505
    Phi_11_200-600m  (lme, Y, year) float64 0.6389 0.634 ... 0.4234 0.4188
    Phi_12_0-200m    (lme, Y, year) float64 4.136 4.047 3.966 ... 2.862 2.844
    Phi_12_200-600m  (lme, Y, year) float64 0.743 0.7369 ... 0.4891 0.4836

In [15]:
#dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_LME_ens_mean_monthly_to_yearly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_Eo_space_LME_ens_mean_monthly_to_yearly.nc', mode='w')